## Easy4PPP tutorials: ppp.ipynb
### Description: A point to point tutorial about how to use core computation libaries and construct the main function of UC-PPP

#### 1. include the Easy4PPP source and dependencies. Attention: "src" needs to be added into system path in this Jupyter Notbook.

In [ ]:
import numpy as np
import os
from tqdm import tqdm
import csv
import sys
sys.path.append("src")
from satpos import *
from sppp import *

#### 2. Set the observation data file path, obervation types, precise orbit and clock products path, antenna file path and output path of solution logs

In [ ]:
#双频非差非组合PPP, 解算文件最小配置(观测值/观测值类型/精密星历文件/精密钟差文件/天线文件/结果输出路径)
obs_file="data/OBS/JFNG/jfng1320.24o"                        #观测值文件, RINEX3及以上
obs_type=['C1C','L1C','D1C','S1C','C2W','L2W','D2W','S2W']   #观测值类型, 以RINEX协议为准
SP3_file="data/Peph_clk/20241320/wum23136.sp3"               #精密星历文件路径
CLK_file="data/Peph_clk/20241320/wum23136.clk"               #精密钟差文件路径
ATX_file="data/ATX/igs20.atx"                                #天线文件, 支持格式转换后的npy文件和IGS天线文件

out_path="nav_result"                                        #导航结果输出文件路径
ion_param=[]                                                 #自定义Klobuchar电离层参数

#### 3. Set the unnecessary file path and solution configs. The details about each variable is listed in UserMannul.pdf

In [ ]:
#可选配置(广播星历文件/DCB改正与产品选项/)
BRDC_file="data/brdc/BRDC00IGS_R_20241320000_01D_MN.rnx"     #广播星历文件, 支持BRDC/RINEX3混合星历
dcb_correction=1                                             #DCB修正选项
dcb_products='CAS'                                           #DCB产品来源, 支持CODE月解文件/CAS日解文件
dcb_file_0="data/DCB/CAS1OPSRAP_20241320000_01D_01D_DCB.BIA" #频间偏差文件, 支持预先转换后的.npy格式
dcb_file_1=""
dcb_file_2=""

obs_start=0                                     #解算初始时刻索引
obs_epoch=0                                     #解算总历元数量
out_age=31                                      #最大容忍失锁阈值时间(单位: s, 用于电离层、模糊度状态重置)
sys_index=['G']                                 #此列表控制obsmat读取范围
sys='GPS'                                       #配置解算系统, 支持GPS/BDS单系统
dy_mode='static'                                #PPP动态模式配置, 支持static, dynamic
f1=1575.42*1e6                                  #配置第一频率
f2=1227.60*1e6                                  #配置第二频率

el_threthod=15.0                                #设置截止高度角
ex_threshold_v=30                               #设置先验残差阈值
ex_threshold_v_sigma=4                          #设置后验残差阈值
Mw_threshold=5.0                                #设置Mw组合周跳检验阈值
GF_threshold=1.0                                #设置GF组合周跳检验阈值

sat_out=[]                                      #设置排除卫星

#### 4.The main function of how to construct PPP computation using Easy4PPP core source codes.

In [ ]:
#CAS DCB产品数据读取
if(dcb_correction==1 and dcb_products=='CAS'):
    dcb_file_0,_=CAS_DCB(dcb_file_0,obs_type[0],obs_type[4])
    dcb_file_1=''       #CAS产品同时包含码间和频间偏差
    dcb_file_2=''
    
#读取观测文件
sys_code=['G','C','E','R']
sys_number=['GPS','BDS','GAL','GLO']
obs_mat=RINEX3_to_obsmat(obs_file,obs_type,sys=sys_code[sys_number.index(sys)],dcb_correction=dcb_correction,dcb_file_0=dcb_file_0,dcb_file_1=dcb_file_1,dcb_file_2=dcb_file_2)
#删除CAS-DCB产品辅助文件
if(dcb_file_0!=""):
    os.unlink(dcb_file_0)

if(not obs_epoch):
    obs_epoch=len(obs_mat)
    print("End index not set, solve all the observations. Total: {}".format(obs_epoch))
    
#读取精密轨道和钟差文件
IGS=getsp3(SP3_file)
clk=getclk(CLK_file)
    
#读取天线文件
try:
    #npy格式
    sat_pcos=np.load(ATX_file,allow_pickle=True)
    sat_pcos=eval(str(sat_pcos))
except:
    #ATX格式
    sat_pcos=RINEX3_to_ATX(ATX_file)
    
#读取广播星历电离层参数
if(not len(ion_param)):
    ion_param=RINEX2ion_params(BRDC_file)
    
#根据配置设置卫星数量
if(sys=='GPS'):
    sat_num=32
elif(sys=='BDS'):
    sat_num=65
else:
    sat_num=0
    

#排除精密星历基准卫星
IGS_PRNS=[list(t.keys())[2:] for t in IGS]
igs_prns=[]
for igs_prn in IGS_PRNS:
    if igs_prn not in igs_prns:
        igs_prns.append(igs_prn)
IGS_PRNS=igs_prns.copy()
if len(IGS_PRNS)-1:
    for i in range(len(IGS_PRNS)-1):
        t_PRNlists=set(IGS_PRNS[i])&set(IGS_PRNS[i+1])
        IGS_PRNS[i+1]=t_PRNlists.copy()
    IGS_PRNS=t_PRNlists.copy()
else:
    IGS_PRNS=IGS_PRNS[0]

for sys in sys_index:
    for prn in range(1,sat_num+1): 
        if("{}{:02d}".format(sys,prn) not in IGS_PRNS):
            sat_out.append("{}{:02d}".format(sys,prn))
print("Satellites outside for no precise eph",sat_out)
    
#初始化PPP滤波器
X,Pk,Qk,GF_sign,Mw_sign,slip_sign,dN_sign,X_time,phase_bias=init_UCPPP(obs_mat,obs_start,IGS,clk,sat_out,ion_param,sat_pcos,sys_sat_num=sat_num,f1=f1,f2=f2)
    
#非差非组合PPP解算
Out_log=UCPPP(obs_mat,obs_start,obs_epoch,IGS,clk,sat_out,ion_param,sat_pcos,
                  el_threthod=el_threthod,ex_threshold_v=ex_threshold_v,ex_threshold_v_sigma=ex_threshold_v_sigma,
                  Mw_threshold=Mw_threshold,GF_threshold=GF_threshold,dy_mode=dy_mode,
                X=X,Pk=Pk,Qk=Qk,X_time=X_time,phase_bias=phase_bias,GF_sign=GF_sign,Mw_sign=Mw_sign,slip_sign=slip_sign,dN_sign=dN_sign,sat_num=sat_num,out_age=out_age,f1=f1,f2=f2)
    
#结果以numpy数组格式保存在指定输出目录下, 若输出目录为空, 则存于nav_result
try:
    np.save(out_path+'/{}.out'.format(os.path.basename(obs_file)),Out_log,allow_pickle=True)
except:
    np.save('nav_result/{}.out'.format(os.path.basename(obs_file)),Out_log,allow_pickle=True)